In [1]:
from pathlib import Path

import polars as pl
import numpy as np

DATA_DIR = Path("../../data/")

### [1693. Daily Leads and Partners](https://leetcode.com/problems/daily-leads-and-partners/)

In [2]:
print(df:=pl.read_csv(DATA_DIR / "1693.csv", dtypes={"date_id": pl.Date}))

shape: (10, 4)
┌────────────┬───────────┬─────────┬────────────┐
│ date_id    ┆ make_name ┆ lead_id ┆ partner_id │
│ ---        ┆ ---       ┆ ---     ┆ ---        │
│ date       ┆ str       ┆ i64     ┆ i64        │
╞════════════╪═══════════╪═════════╪════════════╡
│ 2020-12-08 ┆ toyota    ┆ 0       ┆ 1          │
│ 2020-12-08 ┆ toyota    ┆ 1       ┆ 0          │
│ 2020-12-08 ┆ toyota    ┆ 1       ┆ 2          │
│ 2020-12-07 ┆ toyota    ┆ 0       ┆ 2          │
│ …          ┆ …         ┆ …       ┆ …          │
│ 2020-12-08 ┆ honda     ┆ 2       ┆ 1          │
│ 2020-12-07 ┆ honda     ┆ 0       ┆ 1          │
│ 2020-12-07 ┆ honda     ┆ 1       ┆ 2          │
│ 2020-12-07 ┆ honda     ┆ 2       ┆ 1          │
└────────────┴───────────┴─────────┴────────────┘


MySQL
```sql
select date_id, make_name, 
    count(distinct lead_id) as unique_leads,
    count(distinct partner_id) as unique_partners
from DailySales
group by date_id, make_name
```

In [3]:
print(
    df.group_by("date_id", "make_name")
    .agg(
        pl.n_unique("lead_id").alias("unique_leaders"),
        pl.n_unique("partner_id").alias("unique_partners"))
)

shape: (4, 4)
┌────────────┬───────────┬────────────────┬─────────────────┐
│ date_id    ┆ make_name ┆ unique_leaders ┆ unique_partners │
│ ---        ┆ ---       ┆ ---            ┆ ---             │
│ date       ┆ str       ┆ u32            ┆ u32             │
╞════════════╪═══════════╪════════════════╪═════════════════╡
│ 2020-12-08 ┆ toyota    ┆ 2              ┆ 3               │
│ 2020-12-07 ┆ honda     ┆ 3              ┆ 2               │
│ 2020-12-07 ┆ toyota    ┆ 1              ┆ 2               │
│ 2020-12-08 ┆ honda     ┆ 2              ┆ 2               │
└────────────┴───────────┴────────────────┴─────────────────┘


### [1741. Find Total Time Spent by Each Employee](https://leetcode.com/problems/find-total-time-spent-by-each-employee/)

In [4]:
print(df:=pl.read_csv(DATA_DIR / "1741.csv", dtypes={"event_day": pl.Date}))

shape: (5, 4)
┌────────┬────────────┬─────────┬──────────┐
│ emp_id ┆ event_day  ┆ in_time ┆ out_time │
│ ---    ┆ ---        ┆ ---     ┆ ---      │
│ i64    ┆ date       ┆ i64     ┆ i64      │
╞════════╪════════════╪═════════╪══════════╡
│ 1      ┆ 2020-11-28 ┆ 4       ┆ 32       │
│ 1      ┆ 2020-11-28 ┆ 55      ┆ 200      │
│ 1      ┆ 2020-12-03 ┆ 1       ┆ 42       │
│ 2      ┆ 2020-11-28 ┆ 3       ┆ 33       │
│ 2      ┆ 2020-12-09 ┆ 47      ┆ 74       │
└────────┴────────────┴─────────┴──────────┘


MySQL
```sql
select event_day as day, emp_id, 
    sum(out_time - in_time) as total_time
from Employees
group by emp_id, event_day
```

In [5]:
print(
    df.group_by(pl.col("event_day").alias("day"), "emp_id")
    .agg(tota_time=(pl.col("out_time") - pl.col("in_time")).sum())
)

shape: (4, 3)
┌────────────┬────────┬───────────┐
│ day        ┆ emp_id ┆ tota_time │
│ ---        ┆ ---    ┆ ---       │
│ date       ┆ i64    ┆ i64       │
╞════════════╪════════╪═══════════╡
│ 2020-11-28 ┆ 1      ┆ 173       │
│ 2020-11-28 ┆ 2      ┆ 30        │
│ 2020-12-09 ┆ 2      ┆ 27        │
│ 2020-12-03 ┆ 1      ┆ 41        │
└────────────┴────────┴───────────┘


### [1795. Rearrange Products Table](https://leetcode.com/problems/rearrange-products-table/description/)

In [8]:
print(df:=pl.read_csv(DATA_DIR / "1795.csv", null_values="null"))

shape: (2, 4)
┌────────────┬────────┬────────┬────────┐
│ product_id ┆ store1 ┆ store2 ┆ store3 │
│ ---        ┆ ---    ┆ ---    ┆ ---    │
│ i64        ┆ i64    ┆ i64    ┆ i64    │
╞════════════╪════════╪════════╪════════╡
│ 0          ┆ 95     ┆ 100    ┆ 105    │
│ 1          ┆ 70     ┆ null   ┆ 80     │
└────────────┴────────┴────────┴────────┘


PostgreSQL
```postgresql
select *
from (
    select product_id,
        unnest(array['store1', 'store2', 'store3']) as store,
        unnest(array[store1, store2, store3]) as price
    from Products
    ) as p
where price >= 0
```

In [9]:
print(
    df.melt(id_vars="product_id", variable_name="store", value_name="price")
    .filter(pl.col("price") >= 0)
)

shape: (5, 3)
┌────────────┬────────┬───────┐
│ product_id ┆ store  ┆ price │
│ ---        ┆ ---    ┆ ---   │
│ i64        ┆ str    ┆ i64   │
╞════════════╪════════╪═══════╡
│ 0          ┆ store1 ┆ 95    │
│ 1          ┆ store1 ┆ 70    │
│ 0          ┆ store2 ┆ 100   │
│ 0          ┆ store3 ┆ 105   │
│ 1          ┆ store3 ┆ 80    │
└────────────┴────────┴───────┘


### [1873. Calculate Special Bonus](https://leetcode.com/problems/calculate-special-bonus/)

In [8]:
print(df:=pl.read_csv(DATA_DIR / "1873.csv"))

shape: (5, 3)
┌─────────────┬─────────┬────────┐
│ employee_id ┆ name    ┆ salary │
│ ---         ┆ ---     ┆ ---    │
│ i64         ┆ str     ┆ i64    │
╞═════════════╪═════════╪════════╡
│ 2           ┆ Meir    ┆ 3000   │
│ 3           ┆ Michael ┆ 3800   │
│ 7           ┆ Addilyn ┆ 7400   │
│ 8           ┆ Juan    ┆ 6100   │
│ 9           ┆ Kannon  ┆ 7700   │
└─────────────┴─────────┴────────┘


MySQL
```mysql
select employee_id,
    case when employee_id % 2 and name not like 'M%' then salary
    else 0 end as bonus
from Employees
order by employee_id
```

In [10]:
print(
    df.select("employee_id",
           pl.when(
               pl.col("employee_id")%2 == 1, 
               ~pl.col("name").str.starts_with("M"))
           .then(pl.col("salary"))
           .otherwise(0)
           .alias("bonus"))
    .sort("employee_id")
)

shape: (5, 2)
┌─────────────┬───────┐
│ employee_id ┆ bonus │
│ ---         ┆ ---   │
│ i64         ┆ i64   │
╞═════════════╪═══════╡
│ 2           ┆ 0     │
│ 3           ┆ 0     │
│ 7           ┆ 7400  │
│ 8           ┆ 0     │
│ 9           ┆ 7700  │
└─────────────┴───────┘


### [1890. The Latest Login in 2020](https://leetcode.com/problems/the-latest-login-in-2020/)

In [6]:
print(df:=pl.read_csv(DATA_DIR / "1890.csv", dtypes={"time_stamp": pl.Datetime}))

shape: (9, 2)
┌─────────┬─────────────────────┐
│ user_id ┆ time_stamp          │
│ ---     ┆ ---                 │
│ i64     ┆ datetime[μs]        │
╞═════════╪═════════════════════╡
│ 6       ┆ 2020-06-30 15:06:07 │
│ 6       ┆ 2021-04-21 14:06:06 │
│ 6       ┆ 2019-03-07 00:18:15 │
│ 8       ┆ 2020-02-01 05:10:53 │
│ 8       ┆ 2020-12-30 00:46:50 │
│ 2       ┆ 2020-01-16 02:49:50 │
│ 2       ┆ 2019-08-25 07:59:08 │
│ 14      ┆ 2019-07-14 09:00:00 │
│ 14      ┆ 2021-01-06 11:59:59 │
└─────────┴─────────────────────┘


PostgreSQL
```postgresql
select user_id, max(time_stamp) as last_stamp
from Logins
where extract(year from time_stamp) = 2020
group by user_id
```

In [7]:
print(
    df.filter(pl.col("time_stamp").dt.year() == 2020)
     .group_by("user_id")
     .agg(last_stamp=pl.max("time_stamp"))
)

shape: (3, 2)
┌─────────┬─────────────────────┐
│ user_id ┆ last_stamp          │
│ ---     ┆ ---                 │
│ i64     ┆ datetime[μs]        │
╞═════════╪═════════════════════╡
│ 8       ┆ 2020-12-30 00:46:50 │
│ 2       ┆ 2020-01-16 02:49:50 │
│ 6       ┆ 2020-06-30 15:06:07 │
└─────────┴─────────────────────┘


### [1965. Employees with Missing Information](https://leetcode.com/problems/employees-with-missing-information/)

In [5]:
df_employees = pl.read_csv(DATA_DIR / "1965_employees.csv")
df_salaries = pl.read_csv(DATA_DIR / "1965_salaries.csv")
print(df_employees, df_salaries)

shape: (3, 2)
┌─────────────┬──────────┐
│ employee_id ┆ name     │
│ ---         ┆ ---      │
│ i64         ┆ str      │
╞═════════════╪══════════╡
│ 2           ┆ Crew     │
│ 4           ┆ Haven    │
│ 5           ┆ Kristian │
└─────────────┴──────────┘ shape: (3, 2)
┌─────────────┬────────┐
│ employee_id ┆ salary │
│ ---         ┆ ---    │
│ i64         ┆ i64    │
╞═════════════╪════════╡
│ 5           ┆ 76071  │
│ 1           ┆ 22517  │
│ 4           ┆ 63539  │
└─────────────┴────────┘


PostgreSQL
```postgresql
select employee_id
from Employees e full join Salaries s using (employee_id)
where name is null or salary is null
order by employee_id
```

In [7]:
print(
    df_employees.join(df_salaries, on="employee_id", how="outer")
    .filter(pl.col("name").is_null() | pl.col("salary").is_null())
    .select("employee_id")
    .sort("employee_id")
)

shape: (2, 1)
┌─────────────┐
│ employee_id │
│ ---         │
│ i64         │
╞═════════════╡
│ 1           │
│ 2           │
└─────────────┘
